# Zadanie 1.3

In [12]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [13]:
years = np.array([2000, 2002, 2005, 2007, 2010]).reshape(-1, 1)
percentages = np.array([6.5, 7.0, 7.4, 8.2, 9.0])

X = years
Y = percentages

In [14]:
model = LinearRegression()
model.fit(X, Y)

print(f"Współczynnik kierunkowy (a): {model.coef_[0]:.3f}")
print(f"Wyraz wolny (b): {model.intercept_:.3f}")
print(f"Równanie regresji: y = {model.coef_[0]:.3f}x + {model.intercept_:.3f}")

Współczynnik kierunkowy (a): 0.247
Wyraz wolny (b): -487.834
Równanie regresji: y = 0.247x + -487.834


### Funkcja do przewidywania

In [15]:
def predict_unemployment(year):
    return model.predict([[year]])[0]

## 2. Korzystając z otrzymanego modelu określ, w którym roku procent ten przekroczy 12%.

In [16]:
a = model.coef_[0]  
b = model.intercept_ 

target_year = (12 - b) / a
print(f"Rok w ktorym procent bezrobotnych przekroczy 12%: {round(target_year)}")

Rok w ktorym procent bezrobotnych przekroczy 12%: 2023


In [19]:
print(f"Procent bezrobotnych w rpku 2023: {predict_unemployment(2023)}")

Procent bezrobotnych w rpku 2023: 12.117834394904492


## 3. Przedstawić proces znajdowania regresji liniowej przy użyciu niektórych technik animacji z biblioteki (e.g. matplotlib.pyplot) Pythona.

In [25]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

fig, ax = plt.subplots(figsize=(10, 6))  
ax.scatter(years, percentages, color='green', label='Dane rzeczywiste')  
ax.set_xlabel('Rok')  
ax.set_ylabel('Procent bezrobotnych')  
ax.set_title('Regresja liniowa dla danych o bezrobociu') 
ax.grid(True)  

line, = ax.plot([], [],'o--', lw=2, label='Regresja liniowa', color="red")
ax.legend()

def init():
    line.set_data([], [])
    return line,

def update(frame):
    current_years = years[:frame+1]
    current_percent = percentages[:frame+1]
    
    if len(current_years) > 1: 
        temp_model = LinearRegression()
        temp_model.fit(current_years, current_percent)
        
        x_vals = np.array([min(years), max(years)])
        y_vals = temp_model.predict(x_vals.reshape(-1, 1))
        line.set_data(x_vals, y_vals)
        
        if hasattr(update, 'text'):
            update.text.remove()
        equation = f'y = {temp_model.coef_[0]:.3f}x + {temp_model.intercept_:.3f}'
        update.text = ax.text(0.02, 0.95, equation, transform=ax.transAxes, fontsize=12,
                             bbox=dict(facecolor='white', alpha=0.8))
    
    return line,

ani = FuncAnimation(fig, update, frames=len(years),
                    init_func=init, blit=False, interval=1000)

plt.close()
HTML(ani.to_jshtml())

# Zadanie 2 | Uczenie się z danych

# Zadanie 2.3

In [1]:
def activation(dot_product):
    return 1 if dot_product > 0 else -1

def perceptron_learn(examples, weights, alpha=0.5, max_epochs=10):
    for epoch in range(max_epochs):
        print(f"\nEpoch {epoch + 1}")
        error_made = False
        for x_input, y_expected in examples:
            x_vec = [1] + x_input
            dot = sum(w * x for w, x in zip(weights, x_vec))
            output = activation(dot)
            print(f"  Input: {x_input}, Target: {y_expected}, Output: {output}")
            if output != y_expected:
                error_made = True
                for i in range(len(weights)):
                    delta_w = alpha * (y_expected - output) * x_vec[i]
                    weights[i] += delta_w
                    print(f"    w[{i}] updated by {delta_w:.2f} to {weights[i]:.2f}")
        if not error_made:
            print("  No errors, training completed.")
            break
    return weights

training_data = [
    ([0, 0], -1),
    ([0, 1], -1),
    ([1, 0],  1),
    ([1, 1], -1)
]

initial_weights = [0.5, 0.5, 0.5]

final_weights = perceptron_learn(training_data, initial_weights.copy())

print("\nFinal weights:", final_weights)


Epoch 1
  Input: [0, 0], Target: -1, Output: 1
    w[0] updated by -1.00 to -0.50
    w[1] updated by -0.00 to 0.50
    w[2] updated by -0.00 to 0.50
  Input: [0, 1], Target: -1, Output: -1
  Input: [1, 0], Target: 1, Output: -1
    w[0] updated by 1.00 to 0.50
    w[1] updated by 1.00 to 1.50
    w[2] updated by 0.00 to 0.50
  Input: [1, 1], Target: -1, Output: 1
    w[0] updated by -1.00 to -0.50
    w[1] updated by -1.00 to 0.50
    w[2] updated by -1.00 to -0.50

Epoch 2
  Input: [0, 0], Target: -1, Output: -1
  Input: [0, 1], Target: -1, Output: -1
  Input: [1, 0], Target: 1, Output: -1
    w[0] updated by 1.00 to 0.50
    w[1] updated by 1.00 to 1.50
    w[2] updated by 0.00 to -0.50
  Input: [1, 1], Target: -1, Output: 1
    w[0] updated by -1.00 to -0.50
    w[1] updated by -1.00 to 0.50
    w[2] updated by -1.00 to -1.50

Epoch 3
  Input: [0, 0], Target: -1, Output: -1
  Input: [0, 1], Target: -1, Output: -1
  Input: [1, 0], Target: 1, Output: -1
    w[0] updated by 1.00 to 0

# Zadanie 2.4

In [2]:
import math

# Input data and target
x1, x2 = 0.6, 0.1
target = 1.0

# Initial weights and biases [w_x1, w_x2, bias]
W_hidden = [
    [0.1, -0.2, 0.1],  # h1
    [0.0, 0.2, 0.2],  # h2
    [0.2, -0.4, 0.5]  # h3
]

# Output weights [w_h1, w_h2, w_h3, bias]
W_output = [-0.4, 0.1, 0.6, -0.1]

alpha = 0.1


def sigmoid(z):
    return 1.0 / (1.0 + math.exp(-z))


def sigmoid_derivative(s):
    return s * (1.0 - s)


# Forward Pass
h_vals = []

for i, (wx1, wx2, b) in enumerate(W_hidden, start=1):
    z = wx1 * x1 + wx2 * x2 + b
    h = sigmoid(z)
    h_vals.append(h)
    print(f"h{i} net input: {z:.5f} -> h{i} = {h:.5f}")

z_out = W_output[0] * h_vals[0] + W_output[1] * h_vals[1] + W_output[2] * h_vals[2] + W_output[3]
y = sigmoid(z_out)
print(f"\nOutput neuron net input: {z_out:.5f} -> y = {y:.5f}")

error = target - y
print(f"\nError (t - y) = {target:.5f} - {y:.5f} = {error:.5f}")

# Backpropagation
delta_out = error * sigmoid_derivative(y)
print(f"\nOutput neuron delta = {delta_out:.5f}")

new_W_output = W_output.copy()
for j in range(3):
    grad = alpha * delta_out * h_vals[j]
    new_W_output[j] = W_output[j] + grad
    print(
        f"New w_out[{j + 1}] = {W_output[j]:.5f} + {alpha:.2f}*{delta_out:.5f}*{h_vals[j]:.5f} = {new_W_output[j]:.5f}")

grad_b_out = alpha * delta_out * 1
new_W_output[3] = W_output[3] + grad_b_out
print(f"New output bias = {W_output[3]:.5f} + {alpha:.2f}*{delta_out:.5f}*1 = {new_W_output[3]:.5f}")

deltas_hidden = []
for j in range(3):
    w_oj = W_output[j]
    h = h_vals[j]
    delta_h = delta_out * w_oj * sigmoid_derivative(h)
    deltas_hidden.append(delta_h)
    print(f"\nDelta h{j + 1} = {delta_out:.5f} * {w_oj:.5f} * {h:.5f}*(1-{h:.5f}) = {delta_h:.5f}")

new_W_hidden = [row.copy() for row in W_hidden]
for j in range(3):
    wx1_old, wx2_old, b_old = W_hidden[j]
    delta_h = deltas_hidden[j]

    new_wx1 = wx1_old + alpha * delta_h * x1
    new_wx2 = wx2_old + alpha * delta_h * x2
    new_bh = b_old + alpha * delta_h * 1

    new_W_hidden[j] = [new_wx1, new_wx2, new_bh]
    print(f"\nNew w_h{j + 1},x1 = {wx1_old:.5f} + {alpha:.2f}*{delta_h:.5f}*{x1:.1f} = {new_wx1:.5f}")
    print(f"New w_h{j + 1},x2 = {wx2_old:.5f} + {alpha:.2f}*{delta_h:.5f}*{x2:.1f} = {new_wx2:.5f}")
    print(f"New bias h{j + 1} = {b_old:.5f} + {alpha:.2f}*{delta_h:.5f}*1 = {new_bh:.5f}")

h1 net input: 0.14000 -> h1 = 0.53494
h2 net input: 0.22000 -> h2 = 0.55478
h3 net input: 0.58000 -> h3 = 0.64107

Output neuron net input: 0.12614 -> y = 0.53149

Error (t - y) = 1.00000 - 0.53149 = 0.46851

Output neuron delta = 0.11666
New w_out[1] = -0.40000 + 0.10*0.11666*0.53494 = -0.39376
New w_out[2] = 0.10000 + 0.10*0.11666*0.55478 = 0.10647
New w_out[3] = 0.60000 + 0.10*0.11666*0.64107 = 0.60748
New output bias = -0.10000 + 0.10*0.11666*1 = -0.08833

Delta h1 = 0.11666 * -0.40000 * 0.53494*(1-0.53494) = -0.01161

Delta h2 = 0.11666 * 0.10000 * 0.55478*(1-0.55478) = 0.00288

Delta h3 = 0.11666 * 0.60000 * 0.64107*(1-0.64107) = 0.01611

New w_h1,x1 = 0.10000 + 0.10*-0.01161*0.6 = 0.09930
New w_h1,x2 = -0.20000 + 0.10*-0.01161*0.1 = -0.20012
New bias h1 = 0.10000 + 0.10*-0.01161*1 = 0.09884

New w_h2,x1 = 0.00000 + 0.10*0.00288*0.6 = 0.00017
New w_h2,x2 = 0.20000 + 0.10*0.00288*0.1 = 0.20003
New bias h2 = 0.20000 + 0.10*0.00288*1 = 0.20029

New w_h3,x1 = 0.20000 + 0.10*0.01611*0

# Zadanie 3 | Drzewo decyzyjne